In [326]:
import imp
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from keras.losses import categorical_crossentropy
from keras.activations import Softmax
from keras.optimizers import rmsprop_v2
from keras.metrics import accuracy


In [327]:
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000
DATA_PATH = "/home/sokhorn/sokhorn/Recombo/Dataset/seq2seq/khm-eng/khm.txt"


In [328]:
df = pd.read_csv(DATA_PATH, sep="\t", header=None, names=[
                 "english", "khmer", "_"], usecols=[0, 1])
df.khmer = df.khmer.map(lambda x: "\t" + x + "\n")

In [329]:
input_texts = df.english.values
target_texts = df.khmer.values
input_charaters = set()
target_charaters = set()


In [330]:
for char in input_texts:
    for c in char:
        if c not in input_charaters:
            input_charaters.add(c)

for char in target_texts:
    for c in char:
        if c not in target_charaters:
            target_charaters.add(c)


In [331]:
input_charaters = sorted(list(input_charaters))
target_charaters = sorted(list(target_charaters))
num_encoder_token = len(input_charaters)
num_decoder_token = len(target_charaters)
max_encoder_sequence_len = max([len(x) for x in input_texts])
max_decoder_sequence_len = max([len(x) for x in target_texts])


In [332]:
input_token_index = dict(
    [(c, i) for i, c in enumerate(input_charaters)]
)
target_token_index = dict(
    [(c, i) for i, c in enumerate(target_charaters)]
)
# target_token_index[' '] = max(list(target_token_index.values())) + 1

In [333]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_sequence_len, num_encoder_token), dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_sequence_len, num_decoder_token), dtype='float32'
)
decoder_target_data = np.zeros((
    len(input_texts), max_decoder_sequence_len, num_decoder_token
), dtype='float32')

In [334]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.

    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1
    decoder_target_data[i, t:, target_token_index[' ']] = 1


In [335]:
num_decoder_token

114

In [336]:
# start create LSTM
encoder_input = Input(shape=(None, num_encoder_token))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_input)
encoder_state = [state_h, state_c]


In [337]:
decoder_input = Input(shape=(None, num_decoder_token))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(
    decoder_input, initial_state=encoder_state, )

decoder_dense = Dense(num_decoder_token, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [338]:
model = Model([encoder_input, decoder_input], decoder_outputs)
model.compile(optimizer='rmsprop',
              loss=categorical_crossentropy, metrics=[accuracy]
              )
model.fit([encoder_input_data, decoder_input_data],
          y=decoder_target_data,  batch_size=batch_size,
          epochs=epochs
          )


2022-08-18 14:07:18.705800: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 18156096 exceeds 10% of free system memory.
2022-08-18 14:07:18.722984: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 18156096 exceeds 10% of free system memory.


Epoch 1/100
8/8 [==============================] - 9s 612ms/step - loss: 2.2927 - accuracy: 0.0000e+00
Epoch 2/100
8/8 [==============================] - 5s 610ms/step - loss: 1.2200 - accuracy: 0.0000e+00
Epoch 3/100
8/8 [==============================] - 5s 578ms/step - loss: 1.1636 - accuracy: 0.0000e+00
Epoch 4/100
8/8 [==============================] - 5s 572ms/step - loss: 1.1472 - accuracy: 0.0000e+00
Epoch 5/100
8/8 [==============================] - 4s 535ms/step - loss: 1.1367 - accuracy: 0.0000e+00
Epoch 6/100
8/8 [==============================] - 4s 550ms/step - loss: 1.1314 - accuracy: 0.0000e+00
Epoch 7/100
8/8 [==============================] - 5s 608ms/step - loss: 1.0911 - accuracy: 0.0000e+00
Epoch 8/100


KeyboardInterrupt: 